## Import modules

In [36]:
from collections import OrderedDict
import math
import string
import re
import nltk

## Function : Create unigram dictionary

In [37]:
def add_unigrams(unigrams,unigram):
    for uni in unigrams:
        stri = ''.join(uni)
        if not unigram or stri not in unigram:
            unigram[stri] = 1;
        else:
            unigram[stri] = unigram[stri] + 1

## Function : Create bigram dictionary

In [38]:
def add_bigrams(bigrams,bigram):
    for bi in bigrams:
        stri = ' '.join(bi)
        if not bigram or stri not in bigram:
            bigram[stri] = 1;
        else:
            bigram[stri] = bigram[stri] + 1

## Function : Create trigram dictionary

In [39]:
def add_trigrams(trigrams,trigram):
    for tri in trigrams:
        stri = ' '.join(tri)
        if not trigram or stri not in trigram:
            trigram[stri] = 1;
        else:
            trigram[stri] = trigram[stri] + 1

## Function : Create quadgram dictionary

In [40]:
def add_quadgrams(quadgrams,quadgram):
    for quad in quadgrams:
        stri = ' '.join(quad)
        if not quadgram or stri not in quadgram:
            quadgram[stri] = 1;
        else:
            quadgram[stri] = quadgram[stri] + 1

## Function : Parameter Tuning Using Grid Search

In [41]:
def perform_grid_search(vocab,bi_probab,tri_probab,unigram,bigram,trigram,quadgram,tokens):
    lambda1 = 0
    lambda2 = 0
    lambda3 = 0
    max_score = 0
    i=0
    while i<=1:
        j=0
        while j<=1:
            k=0
            while k<=1:
                if (i+j+k)<=1:
                    probab_dict = {}
                    for quad in quadgram:
                        s = quad.split()
                        w = s[-1]
                        stri = ' '.join(s[0:3])
                        p = perform_interpolation(quad,stri,s,quadgram,trigram,bigram,unigram,tokens,i,j,k,(1-i-j-k))
                        if stri in probab_dict:
                            if w not in probab_dict[stri] :
                                probab_dict[stri][w] = p
                                d = OrderedDict(sorted(probab_dict[stri].items(), key=lambda t: t[1]))
                                probab_dict[stri] = d
                        else:
                            probab_dict[stri] = {}
                            probab_dict[stri][w] = p
                    score = calculate_score('held_out_corpus.txt',bi_probab,tri_probab,probab_dict,vocab)
                    print(i,j,k,score,max_score)
                    if score > max_score:
                        lambda1 = i
                        lambda2 = j
                        lambda3 = k
                        max_score = score
                k = k + 0.1
            j = j + 0.1
        i = i + 0.1
    return (lambda1,lambda2,lambda3,(1-lambda1-lambda2-lambda3))
            

## Function : Perform interpolation

In [42]:
def perform_interpolation(quad,stri,s,quadgram,trigram,bigram,unigram,tokens,lambda1,lambda2,lambda3,lambda4):
    p = ( lambda1 * (quadgram[quad]/trigram[stri])) + (lambda2 * (trigram[' '.join(s[1:4])]/bigram[' '.join(s[1:3])])) + (lambda3 * (bigram[' '.join(s[2:4])]/unigram[str(s[2])])) + (lambda4 * (unigram[str(s[3])]/tokens))
    return p

## Function : Create bigram probability dictionary

In [43]:
def add_bi_probability(unigram,bigram,bi_probab):
    for bi in bigram:
        s = bi.split()
        w = s[1]
        stri = s[0]
        p = float(bigram[bi])/unigram[stri]
        if stri in bi_probab:
            if w not in bi_probab[stri]:
                bi_probab[stri][w] = p
                d = OrderedDict(sorted(bi_probab[stri].items(), key=lambda t: t[1]))
                bi_probab[stri] = d
        else:
            bi_probab[stri] = {}
            bi_probab[stri][w] = p

## Function : Create trigram probability dictionary

In [44]:
def add_tri_probability(bigram,trigram,tri_probab):
    for tri in trigram:
        s = tri.split()
        w = s[-1]
        stri = ' '.join(s[0:2])
        p = float(trigram[tri])/bigram[stri]
        if stri in tri_probab:
            if w not in tri_probab[stri]:
                tri_probab[stri][w] = p
                d = OrderedDict(sorted(tri_probab[stri].items(), key=lambda t: t[1]))
                tri_probab[stri] = d
        else:
            tri_probab[stri] = {}
            tri_probab[stri][w] = p

## Function : Create quadgram probability dictionary

In [45]:
def add_probability(vocab,bi_probab,tri_probab,unigram,bigram,trigram,quadgram,probab,tokens):
    #(lambda1,lambda2,lambda3,lambda4) = perform_grid_search(vocab,bi_probab,tri_probab,unigram,bigram,trigram,quadgram,tokens)
    lambda1 = 0.25
    lambda2 = 0.25
    lambda3 = 0.25
    lambda4 = 0.25
    for quad in quadgram:
        s = quad.split()
        w = s[-1]
        stri = ' '.join(s[0:3])
        p = perform_interpolation(quad,stri,s,quadgram,trigram,bigram,unigram,tokens,lambda1,lambda2,lambda3,lambda4)
        if stri in probab:
            if w not in probab[stri]:
                probab[stri][w] = p
                d = OrderedDict(sorted(probab[stri].items(), key=lambda t: t[1]))
                probab[stri] = d
        else:
            probab[stri] = {}
            probab[stri][w] = p

## Function : Create vocabulary list

In [46]:
def create_token_list(tokens,vocab):
    newlist=[]
    for word in tokens:
        if word:
            if not vocab or word not in vocab: 
                vocab[word] = len(vocab)
            newlist.append(str(vocab[word]))
    return newlist

## Function : Remove punctuations

In [47]:
def remove_punc(text):
    for c in string.punctuation:
        if c!='\'':
            text=text.replace(c," ")
    text = text.replace("' "," ")
    text = text.replace("\n"," ")
    answer = text.lower()
    return answer

## Function : Tokenize the corpus

In [48]:
def tokenize(text, ngrams,vocab):
    clean_text = remove_punc(text)
    tokens = create_token_list(clean_text.split(),vocab)
    #print(tokens)
    return [tuple(tokens[i:i+ngrams]) for i in range(len(tokens)-ngrams+1)]

## Function : Predict word using backoff

In [49]:
def predictWord(s,bi_probab,tri_probab,probab,vocab,rank = 0):
    predict = ""
    l = s.split()
    tri = ' '.join(s[1:3])
    bi = s[2]
    if s in probab:
        predict = list(probab[s].keys())[-1 - rank]
    elif tri in tri_probab:
        predict = list(tri_probab[tri].keys())[-1 - rank]
    elif bi in bi_probab:
        predict = list(bi_probab[bi].keys())[-1 - rank]
    return predict

## Preprocessing performed

In [50]:
def train_data(unigram,bigram,trigram,quadgram,vocab):
    l = 0
    with open('training_corpus.txt',buffering=20000,encoding='latin1') as f:
        for line in f:
            data = tokenize(line,1,vocab)
            add_unigrams(data,unigram)
            l = l + len(data)
            data = tokenize(line,2,vocab)
            add_bigrams(data,bigram)
            data = tokenize(line,3,vocab)
            add_trigrams(data,trigram)
            data = tokenize(line,4,vocab)
            add_quadgrams(data,quadgram)
    f.close()
    return l

## Function : Calculate score for model

In [51]:
def calculate_score(file_name,bi_probab,tri_probab,probab,vocab):
    score = 0
    l = list(vocab.keys())
    with open(file_name,buffering=20000,encoding='latin1') as f:
        for line in f:
            text = remove_punc(line)
            tokens = text.split()
            data = [tuple(tokens[i:i+4]) for i in range(len(tokens)-4+1)]            
            for quad in data:
                numbers = []
                flag = 0
                for ele in quad:
                    if ele in vocab:
                        numbers.append(str(vocab[ele]))
                    else:
                        flag = 1
                        break
                if flag == 0:
                    tri = ' '.join(numbers).split()
                    w = tri[-1]
                    del tri[-1]
                    s = ' '.join(tri)
                    predict = predictWord(s,bi_probab,tri_probab,probab,vocab)
                    if w == predict:
                        score = score + 1
                        #print(score,',',w,',',predict)
                        #print(l[(int)(numbers[0])],' ',l[(int)(numbers[1])],' ',l[(int)(numbers[2])],' ',l[(int)(numbers[3])])
    f.close()
    return score

## Function : Calculate Perplexity using Good-Turing Smoothing

In [52]:
def calculate_perplexity_good_turing(n,vocab,quadgram):
    freq = {}
    for quad in quadgram:
        if quadgram[quad] not in freq:
            val = 0
            for four in quadgram:
                if quadgram[quad] == quadgram[four]:
                    val = val +1
            freq[quadgram[quad]] = val
   
    x_mean = 0
    y_mean = 0
    
    for item in freq:
        x_mean = x_mean + math.log(item)
        y_mean = y_mean + math.log(freq[item])
    
    x_mean = x_mean/len(freq)
    y_mean = y_mean/len(freq)
    
    num = 0
    denom = 0
    for item in freq:
        x = math.log(item)
        y = math.log(freq[item])
        num = num + ((x - x_mean)*(y - y_mean))
        denom = denom + math.pow((x - x_mean),2)
    
    b = num/denom
    log_a = y_mean - (b * x_mean)
    
    p = 1.0
    N = 0
    i = 0
    N_zero = 0
    with open('testing_corpus.txt',buffering=20000,encoding='latin1') as f:
        for line in f:
            text = remove_punc(line)
            tokens = text.split()
            data = [tuple(tokens[i:i+4]) for i in range(len(tokens)-4+1)] 
            for quad in data:
                c = 0
                numbers = []
                flag = 0
                for ele in quad:
                    if ele in vocab:
                        numbers.append(str(vocab[ele]))
                    else:
                        flag = 1
                        break
                if flag == 0:
                    four = ' '.join(numbers)
                    if four in quadgram:
                        c = quadgram[four]
                N_c_plus_one = math.exp(log_a + (b * math.log(c+1)))
                if c==0:
                    N_zero = N_zero + 1
                    N_c = N_zero
                else:
                    N_c = math.exp(log_a + (b * math.log(c)))
                c_star = float((c+1) * N_c_plus_one)/N_c
                p = p * ((1/float(c_star)) ** (1.0/n))
                N = N + (N_c * c_star)
                i = i + 1
    p = p * (N ** (float(i)/n))
    print("Perplexity using Good-Turing Smoothing : ",p)            
            

## Function : Calculate Perplexity using Kneser-Ney Smoothing

In [56]:
def calculate_perplexity_kneser_ney(n,vocab,unigram,bigram,trigram,quadgram):
    p = 1.0
    i = 0
    with open('testing_corpus.txt',buffering=20000,encoding='latin1') as f:
        for line in f:
            if i == 1000:
                break;
            i = i + 1
            text = remove_punc(line)
            tokens = text.split()
            data = [tuple(tokens[i:i+4]) for i in range(len(tokens)-4+1)] 
            for quad in data:
                numbers = []
                flag = 0
                for ele in quad:
                    if ele in vocab:
                        numbers.append(str(vocab[ele]))
                    else:
                        flag = 1
                        break
                if flag == 0:
                    D = 0.75
                    num = 0
                    val = 0
                    w4 = numbers[3]
                    w3 = numbers[2]
                    for item in bigram:
                        if item.split()[1] == w4:
                            num = num + 1
                        if item.split()[0] == w3:
                            val = val + 1
                    pkn4 = float(num)/len(bigram)
                    pkn3 = (pkn4 * D * val)/float(unigram[w3])
                    
                    if ' '.join(numbers[2:4]) in bigram:
                        pkn3 = pkn3 + (max(bigram[' '.join(numbers[2:4])] - D,0)/float(unigram[w3]))
                    val = 0
                    for item in trigram:
                        if ' '.join(item.split()[0:2]) == ' '.join(numbers[1:3]):
                            val = val + 1
                    pkn2 = 0
                    if ' '.join(numbers[1:3]) in bigram:
                        pkn2 = (D * val * pkn3)/float(bigram[' '.join(numbers[1:3])])
                        if ' '.join(numbers[1:4]) in trigram:
                            pkn2 = pkn2 + (max(trigram[' '.join(numbers[1:4])] - D,0)/float(bigram[' '.join(numbers[1:3])]))
                    pkn1 = 0
                    val = 0
                    for item in quadgram:
                        if ' '.join(item.split()[0:3]) == ' '.join(numbers[0:3]):
                            val = val + 1
                    if ' '.join(numbers[0:3]) in trigram:
                        pkn1 = (D * val * pkn2)/float(trigram[' '.join(numbers[0:3])])
                        if ' '.join(numbers) in quadgram:
                            pkn1 = pkn1 + (max(quadgram[' '.join(numbers)] - D,0)/float(trigram[' '.join(numbers[0:3])]))
                    if pkn1 != 0:
                        p = p * (1.0/(pkn1 ** (1.0/n)))
    print("Perplexity using Kneser-Ney Smoothing : ",p)
                                    
                    
                    

## Function : Calculate Perplexity using Add-1 Smoothing

In [54]:
def calculate_perplexity_add_one(n,vocab,trigram,quadgram):
    p = 1.0
    with open('testing_corpus.txt',buffering=20000,encoding='latin1') as f:
        for line in f:
            text = remove_punc(line)
            tokens = text.split()
            data = [tuple(tokens[i:i+4]) for i in range(len(tokens)-4+1)] 
            for quad in data:
                count_quadgrams = 1
                count_trigrams = len(quadgram)
                numbers = []
                flag = 0
                for ele in quad:
                    if ele in vocab:
                        numbers.append(str(vocab[ele]))
                    else:
                        flag = 1
                        break
                if flag == 0:
                    four = ' '.join(numbers)
                    three = ' '.join(numbers[0:3])
                    if three in trigram:
                        count_trigrams = count_trigrams + trigram[three]
                    if four in quadgram:
                        count_quadgrams = count_quadgrams + quadgram[four]
                p=p*(((1/float(count_quadgrams))*count_trigrams)**(1./n))
    print("Perplexity after Add-1 Smoothing: ",p)        
    

## Run the Language Model

In [57]:
#%%timeit
def main():
    vocab = {}
    probab = {}
    tri_probab = {}
    bi_probab = {}
    unigram = {}
    bigram = {}
    trigram = {}
    quadgram = {}
    n = train_data(unigram,bigram,trigram,quadgram,vocab)
    add_bi_probability(unigram,bigram,bi_probab)
    add_tri_probability(bigram,trigram,tri_probab)
    add_probability(vocab,bi_probab,tri_probab,unigram,bigram,trigram,quadgram,probab,n)
    print("Score : ",calculate_score('testing_corpus.txt',bi_probab,tri_probab,probab,vocab))
    calculate_perplexity_add_one(n,vocab,trigram,quadgram)
    calculate_perplexity_good_turing(n,vocab,quadgram)
    calculate_perplexity_kneser_ney(n,vocab,unigram,bigram,trigram,quadgram)
if __name__== "__main__":
  main()

Score :  2812
Perplexity after Add-1 Smoothing:  2.638884919558617
Perplexity using Good-Turing Smoothing :  4.774585341840012
Perplexity using Kneser-Ney Smoothing :  1.003290593918522
